In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import agnostic_control as ac

In [ ]:
grid = ac.Grid()
avals = [-2, 1]
probs = [0.8, 0.2]
prior = ac.Prior(avals, probs)

In [ ]:
nsteps = 200
tmax = 1.0
J_expected = ac.expected_cost(grid, prior, nsteps, tmax)
u_opt = ac.optimal_control(grid, J_expected)

In [ ]:
origin = grid.origin
plt.plot(grid.q, J_expected[0,:,origin[1],origin[2]])

In [ ]:
def calculate_regret(u, a_vals):
    return ac.calculate_regret_vals(grid, tmax, u, a_vals)

def plot_regret(prior, a_vals, regret_vals):
    plt.plot(a_vals, regret_vals, lw=3)
    plt.axvline(prior.a[0], lw=3, ls='--', c='k', label="Location of candidate $a$'s in prior")
    plt.axvline(prior.a[1], lw=3, ls='--', c='k')
    plt.legend()
    plt.xlabel('Value of $a$')
    plt.ylabel('Hybrid regret')
    plt.title(f"Maximum regret: {max(regret_vals):.3f}\n" +
              rf"with prior $a$ = [{prior.a[0]:.3f}, {prior.a[1]:.3f}], $p$ = [{prior.p[0]:.3f}, {prior.p[1]:.3f}]")

In [ ]:
a_vals = np.linspace(-3, 1, 32)
regret_vals = calculate_regret(u_opt, a_vals)

In [ ]:
plot_regret(prior, a_vals, regret_vals)

In [ ]:
prior2 = ac.Prior([-2, 1], [0.2, 0.8])
J_expected2 = ac.expected_cost(grid, prior2, nsteps, tmax)
u_opt2 = ac.optimal_control(grid, J_expected2)

In [ ]:
regret_vals2 = calculate_regret(u_opt2, a_vals)

In [ ]:
plot_regret(prior, a_vals, regret_vals)
plot_regret(prior2, a_vals, regret_vals2)

In [ ]:
new_prior = ac.optimize_prior(grid, prior, nsteps, tmax, fixed_inds=[1,])

In [ ]:
J_expected3 = ac.expected_cost(grid, new_prior, nsteps, tmax)
u_opt3 = ac.optimal_control(grid, J_expected3)
regret_vals3 = calculate_regret(u_opt3, a_vals)
plot_regret(new_prior, a_vals, regret_vals3)

### Compare with previous code

In [ ]:
import PDESolver as tp2

In [ ]:
q0   = 0.0 # starting position of the state
lam  = 1.0 # redundant

tmin = 0
tmax = 1
tpts = int(tmax*200)+1

# Example prior
avec   = np.array([-2, 1.])
arho   = np.array([0.81  ,0.19])

#Simulation parameters
params = tp2.Parameters(tmin,tmax,avec,arho,q0,lam,a=None)

#Grid for simulation
grid2 = tp2.Grid(51,-np.pi,np.pi,27,-4.,4.,26,0.,8.,tpts,tmin,tmax)

In [ ]:
Sh, S, S0 = tp2.SingleSolve(avec, arho, grid2, params)

In [ ]:
params.a

In [ ]:
# compare Sh (expected cost for given prior) with J_expected
origin = grid.origin
tstep = 0
nx1 = origin[1]
# nx1 = 17
plt.plot(grid.q, J_expected[tstep, :, nx1, origin[2]])
plt.plot(grid.q, Sh[:, nx1, origin[2]],'--')

In [ ]:
J_known0 = ac.cost_known_a(grid, nsteps, tmax, u_opt, a=0)

In [ ]:
# compare S (cost, for given a, of Bayesian strategy) with J_known0
nx1 = origin[1]
plt.plot(grid.q, J_known0[:, nx1, origin[2]])
plt.plot(grid.q, S[:, nx1, origin[2]], '--')

In [ ]:
newton = tp2.NewtonSolver(1e-4, 1e-3)

In [ ]:
avec4, arho4, regret4 = newton.Solver(grid2, avec, arho, params)

In [ ]:
print(avec4)
print(arho4)
print(regret4)

In [ ]:
# compare rhs values at t = tmax
t = 1.0
dt = 1 / nsteps
tstep = 100
t = tstep * dt
origin = grid.origin

# Clancy's code
prior3 = ac.Prior(params.avec, params.arho)
abar = ac.calculate_abar(grid, prior3, t)
rhs1 = ac.rhs_expected_cost(grid, prior3, J_expected[tstep], t)

# Max's code
rhs_obj = tp2.RHS()
field = tp2.Field(grid2, params)
abar2 = field.abar(t)
S = np.zeros(grid.shape)
rhs_Sh, rhs_S = rhs_obj.rhs_Sh_S(S, J_expected[tstep], abar2, grid2, params)

In [ ]:
# compare right-hand sides, as a function of q
plt.plot(grid.q[1:-1], rhs1[1:-1,::2,origin[2]],'-x');
plt.plot(grid.q[1:-1], rhs_Sh[1:-1,::2,origin[2]],'--');
plt.xlabel("q")
plt.title("Right-hand side at x2 = 0")

In [ ]:
plt.plot(grid.q, abar[:,origin[1],origin[2]], '-x')
plt.xlabel('q')
plt.ylabel(r'$\bar a$')
plt.title(f"t = {t:.2f}");

In [ ]:
plt.plot(grid.x1, abar[0,:,origin[2]], '-x', label=f"q = {grid.q[0]:.2f}")
plt.plot(grid.x1, abar[origin[0],:,origin[2]], '-x', label=f"q = {grid.q[origin[0]]:.2f}")
plt.plot(grid.x1, abar[-1,:,origin[2]], '-x', label=f"q = {grid.q[-1]:.2f}")
plt.xlabel(r'$\zeta_1$')
plt.ylabel(r'$\bar a$')
plt.legend()
plt.title(f"t = {t:.2f}");

In [ ]:
plt.plot(grid.x2, abar[0,origin[1],:], '-x', label=f"q = {grid.q[0]:.2f}")
plt.plot(grid.x2, abar[origin[0],origin[1],:], '-x', label=f"q = {grid.q[origin[0]]:.2f}")
plt.plot(grid.x2, abar[-1,origin[1],:], '-x', label=f"q = {grid.q[-1]:.2f}")
plt.xlabel(r'$\zeta_2$')
plt.ylabel(r'$\bar a$')
plt.title(f"t = {t:.2f}")
plt.legend();